In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
import pathlib

# pathlib.Path(__file__).parent.resolve()
polls = pd.read_csv("../data/polls_synthetic.csv")
# ratings = pd.read_csv("ratings.csv")

#polls

In [3]:
topics = ["Tech",
          "Sport",
          "Politics",
          "Science",
          "Activity",
          "Art",
          "Beauty",
          "Economy",
          "Fashion",
          "Food&Drinks",
          "Fun",
          "Health",
          "Music",
          "Pets",
          "Travel",
          "Movies & TV shows",
          "Home Décor"
          ]


polls_list = polls["topic"].str.get_dummies(sep="|")



#print(len(polls_list.columns), polls_list.columns)

#print( [item for item in polls_list.columns if item not in topics] )
 
# Concatenate the binary genre columns back to the 'movies' DataFrame
polls = pd.concat([polls, polls_list], axis=1)
#polls

In [12]:
#polls.set_index('poll_ID', inplace=True)

#polls = polls.reset_index() 
#polls


In [13]:
for i in range(len(polls)):
    if not isinstance(polls.iloc[i, 4], str):
        print(polls.iloc[i, 0], polls.iloc[i, 1], polls.iloc[i, 2], polls.iloc[i, 3], polls.iloc[i, 4])


In [14]:
tf_idf = TfidfVectorizer(stop_words="english")
tf_idf_matrix = tf_idf.fit_transform(polls["topic"])

In [15]:
cosine_similarity_matrix = cosine_similarity(tf_idf_matrix, tf_idf_matrix)

In [19]:
def idx_from_title(df, title):
    return df[df["title"] == title].index.values[0]


# idx_from_title(movies, "Sabrina (1995)")


def title_from_idx(df, idx):
    return df[df.index == idx]


title_from_idx(polls, 23)

,poll_ID,author_ID,title,option,topic,Activity,Art,Beauty,Economy,Fashion,...,Health,Home Décor,Movies & TV shows,Music,Pets,Politics,Science,Sport,Tech,Travel
23,23,8,Should there be stricter regulations on social...,Agree|Disagree|Undecided,Politics,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [17]:
def recommendations(
    original_title, df, cosine_similarity_matrix, number_of_recommendations
):
    index = idx_from_title(df, original_title)
    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    similarity_scores_sorted = sorted(
        similarity_scores, key=lambda x: x[1], reverse=True
    )
    recommendations_indices = [
        t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    ]
    return df["title"].iloc[recommendations_indices]

In [20]:
# recommendations('Toy Story (1995)', movies, cosine_similarity_matrix, 10)
#index = idx_from_title(polls, "Toy Story (1995)")
index = 23


similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
recommendations_indices = [t[0] for t in similarity_scores_sorted[1 : (10 + 1)]]
polls["title"].iloc[recommendations_indices]

21            What is your opinion on healthcare reform?
22     Is climate change a top priority for the gover...
23     Should there be stricter regulations on social...
24        Do you believe in term limits for politicians?
25     Is foreign aid beneficial for diplomatic relat...
26               Should the voting age be lowered to 16?
27       Do you think campaign finance laws need reform?
28          Is the current immigration system effective?
29     Should the government invest more in renewable...
188    Should Governments Invest More in Space Explor...
Name: title, dtype: object

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
from collections import Counter


pd.set_option("max_colwidth", None)


""" def get_polls_list(polls_csv_path="/data/polls_synthetic.csv"):
    path = Path(__file__).parent.parent.resolve()
    polls = pd.read_csv(str(path) + polls_csv_path)
    return polls """


def get_polls_list(polls_json_path="/data/elas_polls.json"):
    path = Path(__file__).parent.parent.resolve()
    polls = pd.read_json(str(path) + polls_json_path)
    print(polls)
    return polls


def encode_topics(df):
    topics = df["topic"].str.get_dummies(sep="|")
    df = pd.concat([df, topics], axis=1)
    return df


def set_index(df, index_column="poll_ID"):
    df.set_index(index_column, inplace=True)
    return df


def reset_index(df):
    df.reset_index()
    return df


def check_column_type(df, column_name, check_type):
    column_index = df.columns.get_loc(column_name)
    for i in range(len(df)):
        if not isinstance(df.iloc[i, column_index], check_type):
            print(
                f"error: {df.iloc[i, 0], df.iloc[i, 1],df.iloc[i, 2], df.iloc[i, 3], df.iloc[i, 4]}"
            )


def create_tf_idf_matrix(df, column):
    tf_idf = TfidfVectorizer(stop_words="english")
    tf_idf_matrix = tf_idf.fit_transform(df[column])
    return tf_idf_matrix


def calc_cosine_similarity_matrix(tf_idf_matrix_1, tf_idf_matrix_2):
    cosine_similarity_matrix = cosine_similarity(tf_idf_matrix_1, tf_idf_matrix_2)
    return cosine_similarity_matrix


def idx_from_title(df, title):
    return df[df["title"] == title].index.values[0]


def title_from_idx(df, idx):
    return df[df.index == idx]


def gen_recommendations(index, df, cosine_similarity_matrix, number_of_recommendations):
    # index = idx_from_title(df, original_title)
    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    similarity_scores_sorted = sorted(
        similarity_scores, key=lambda x: x[1], reverse=True
    )

    recommendations_indices = [
        t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    ]
    recommendations = list(df["title"].iloc[recommendations_indices])
    # print(recommendations)
    # print(similarity_scores_sorted, type(similarity_scores_sorted))
    # recommendations_indices = [
    #    t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    # ]
    # recommendations_scores = [
    #    t[1] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
    # ]
    # return (df["title"].iloc[recommendations_indices], recommendations_scores)

    return recommendations


def gen_rec_from_list_of_polls(
    interacted_polls, polls, cosine_similarity_matrix, number_of_recommendations
):
    recommendations = []
    for poll_id in interacted_polls:
        # index = idx_from_title(df, original_title)
        similarity_scores = list(enumerate(cosine_similarity_matrix[poll_id]))
        similarity_scores_sorted = sorted(
            similarity_scores, key=lambda x: x[1], reverse=True
        )

        recommendations_indices = [
            t[0] for t in similarity_scores_sorted[1 : (number_of_recommendations + 1)]
        ]
        recs = list(polls["poll_ID"].iloc[recommendations_indices])
        # print(f"recommended polls for {poll_id} are:{recs}")
        recommendations.append(recs)

    flattened_recommendations = [
        item for sublist in recommendations for item in sublist
    ]
    flattened_recommendations = Counter(flattened_recommendations)
    n_most_recommended = flattened_recommendations.most_common(
        number_of_recommendations
    )
    n_most_recommended = [t[0] for t in n_most_recommended]
    print(n_most_recommended)

    return n_most_recommended


if __name__ == "__main__":
    polls = get_polls_list("/data/polls_synthetic.csv")
    
    #polls = encode_topics(polls)
    #
    #check_column_type(polls, 4, str)
    #tf_idf_matrix = create_tf_idf_matrix(polls, "title")
    #cosine_similarity_matrix = calc_cosine_similarity_matrix(
    #    tf_idf_matrix, tf_idf_matrix
    #)
#
    #liked_poll_title = "Do you think cryptocurrency is the future of finance?"
    #liked_poll_index = idx_from_title(polls, liked_poll_title)
#
    #liked_polls = []
    #recommended_list = recommendations(
    #    liked_poll_index, polls, cosine_similarity_matrix, 10
    #)
    #print(f"recommended_list: [{recommended_list}]")
#
    ## print(f"liked poll: [{liked_poll_title}] \nrecommended polls: \n{recommended_list}")
#